In [7]:
import rdkit
from rdkit import Chem
from rdkit.Chem import rdForceFieldHelpers, AllChem
from rdkit.Chem import SDMolSupplier
from rdkit.Chem import PDBWriter

def load_molecule(file_path):
    """Load a molecule from a file (SDF or PDB) and add explicit hydrogens."""
    if file_path.lower().endswith('.sdf'):
        suppl = SDMolSupplier(file_path, removeHs=False)
        mol = next(suppl)
    elif file_path.lower().endswith('.pdb'):
        mol = Chem.MolFromPDBFile(file_path, removeHs=False)
    else:
        raise ValueError("Unsupported file format. Please use .sdf or .pdb")
    
    if mol is None:
        raise ValueError(f"Failed to load molecule from {file_path}")
    
    mol = Chem.AddHs(mol)
    return mol

def prepare_molecule(mol):
    """Prepare the molecule for force field calculations."""
    mol = Chem.AddHs(mol)
    if not mol.GetNumConformers():
        AllChem.EmbedMolecule(mol, randomSeed=42)
    return mol

def minimize_ligand(ligand, protein, max_iters=500, conf_id=-1):
    """
    Minimize the ligand using MMFF forcefield while keeping the protein fixed.
    
    :param ligand: RDKit Mol object of the ligand
    :param protein: RDKit Mol object of the protein
    :param max_iters: Maximum number of iterations for minimization
    :param conf_id: ID of the conformer to minimize (-1 for the first one)
    :return: Minimized ligand as RDKit Mol object
    """
    # Prepare molecules
    ligand = prepare_molecule(ligand)
    protein = prepare_molecule(protein)
    
    # Create a combined molecule
    combined = Chem.CombineMols(ligand, protein)
    
    try:
        # Set up MMFF properties
        # mp = rdForceFieldHelpers.UFFGetMoleculeProperties(combined)
        # if mp is None:
        #     raise ValueError("Failed to set up MMFF properties. Check your input molecules.")
        
        # Create MMFF force field
        ff = rdForceFieldHelpers.UFFGetMoleculeForceField(combined, confId=0, ignoreInterfragInteractions=False)
        if ff is None:
            raise ValueError("Failed to create force field. Check your input molecules.")
        
        # Fix the protein atoms
        for atom in combined.GetAtoms():
            if atom.GetIdx() >= ligand.GetNumAtoms():  # These are protein atoms
                ff.AddFixedPoint(atom.GetIdx())
        
        # Run the minimization
        result = ff.Minimize(maxIts=10, energyTol=1e-4, forceTol=1e-3)
        
        if result == 1:
            print(f"Minimization did not converge in {max_iters} iterations.")
        elif result == -1:
            print("Minimization failed.")
        else:
            print("Minimization successful.")
        
        # Extract the minimized ligand
        minimized_ligand = Chem.Mol(ligand)
        conf = minimized_ligand.GetConformer()
        for i in range(ligand.GetNumAtoms()):
            conf.SetAtomPosition(i, combined.GetConformer().GetAtomPosition(i))
        
        return minimized_ligand
    
    except ValueError as e:
        print(f"Error during minimization: {str(e)}")
        return None

In [8]:
ligand_file = "/home/tony/DockM8_v2/tests/test_files/fabind_test/fabind_plus/FCG18472628.sdf"
protein_file = "/home/tony/DockM8_v2/tests/test_files/fabind_test/fabind_plus/pdb/4kd1_p.pdb"

ligand = load_molecule(ligand_file)
protein = load_molecule(protein_file)

minimized_ligand = minimize_ligand(ligand, protein)

# Save the minimized ligand
writer = Chem.SDWriter("/home/tony/DockM8_v2/tests/test_files/fabind_test/fabind_plusminimized_ligand.sdf")
writer.write(minimized_ligand)
writer.close()

print("Minimized ligand saved as 'minimized_ligand.sdf'")

[22:58:16] 

****
Pre-condition Violation
RingInfo not initialized
Violation occurred on line 28 in file /home/conda/feedstock_root/build_artifacts/rdkit_1711521244359/work/Code/GraphMol/RingInfo.cpp
Failed Expression: df_init
----------
Stacktrace:
 0# Invar::Invariant::toString[abi:cxx11]() const in /home/tony/miniconda/envs/dockm8/lib/python3.10/site-packages/rdkit/../../../libRDKitRDGeneral.so.1
 1# Invar::operator<<(std::ostream&, Invar::Invariant const&) in /home/tony/miniconda/envs/dockm8/lib/python3.10/site-packages/rdkit/../../../libRDKitRDGeneral.so.1
 2# RDKit::RingInfo::isAtomInRingOfSize(unsigned int, unsigned int) const in /home/tony/miniconda/envs/dockm8/lib/python3.10/site-packages/rdkit/Chem/../../../../libRDKitGraphMol.so.1
 3# RDKit::UFF::Tools::addAngles(RDKit::ROMol const&, std::vector<ForceFields::UFF::AtomicParams const*, std::allocator<ForceFields::UFF::AtomicParams const*> > const&, ForceFields::ForceField*) in /home/tony/miniconda/envs/dockm8/lib/python3.10/si

RuntimeError: Pre-condition Violation
	RingInfo not initialized
	Violation occurred on line 28 in file Code/GraphMol/RingInfo.cpp
	Failed Expression: df_init
	RDKIT: 2023.09.6
	BOOST: 1_84
